In [ ]:
!conda install -y gdown

/bin/bash: line 1: conda: command not found


In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1WetCAaP_ebbcXTVd-tCcwqzJggzEupi3'
output = 'data.csv'
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1WetCAaP_ebbcXTVd-tCcwqzJggzEupi3
To: /content/data.csv
100%|██████████| 3.82M/3.82M [00:00<00:00, 29.0MB/s]


'data.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
import math
import itertools

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
df_raw = pd.read_csv('data.csv', index_col = 0)
try:
  df_raw = df_raw.set_index('SEQN', drop=True)
  df_raw = df_raw.drop(columns = "Unnamed: 0")
except:
  pass

df_raw.head()

,Dieta1_DR1TKCAL,Exami2_BMXBMI,Dieta1_DR1TCARB,Dieta1_DR1TFIBE,Dieta1_DR1TSFAT,Dieta1_DR1TTFAT,Dieta1_DR1TPFAT,Dieta1_DR1TCALC,Quest6_DED125,Dieta1_DR1TMFAT,...,Labor2_URDFLOW1,Quest17_DPQ040,Demog1_DMDHHSZE,Quest16_MCQ010,Quest16_MCQ160B,Quest16_MCQ220,Quest16_MCQ300C,Quest16_MCQ300A,Quest16_MCQ366A,Quest16_MCQ366B
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,0,0,1,1,1,0,1,2,1,1,...,0.895435,9.000000e+00,5.397605e-79,2.0,1.0,9.0,9.0,9.0,9.0,9.0
93704.0,0,0,0,0,0,0,1,1,1,0,...,0.895435,9.000000e+00,5.397605e-79,2.0,1.0,9.0,9.0,9.0,9.0,9.0
93705.0,0,3,0,0,1,1,2,0,1,1,...,1.204000,5.397605e-79,1.000000e+00,1.0,2.0,2.0,1.0,2.0,2.0,2.0
93706.0,2,1,0,0,3,3,3,1,2,3,...,0.783489,5.397605e-79,1.000000e+00,2.0,1.0,9.0,9.0,9.0,2.0,2.0
93707.0,1,0,1,1,3,2,1,0,1,3,...,0.526000,9.000000e+00,5.397605e-79,2.0,1.0,9.0,9.0,9.0,9.0,9.0


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def prep_data(df):
  X = df.copy().drop(columns=['Quest16_MCQ160B'])
  y = df['Quest16_MCQ160B']

  y = y.replace({2: 0})
  y = y.replace({9: 0})

  y = y.astype(int)

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

  scaler = StandardScaler()
  X_train.iloc[:, 28:] = scaler.fit_transform(X_train.iloc[:, 28:])
  X_val.iloc[:, 28:] = scaler.transform(X_val.iloc[:, 28:])

  X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
  X_val = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)

  print("Train: ", X_train.shape, " ", y_train.shape)
  print("Val: ", X_val.shape, " ", y_val.shape)
  print("Column Used: ", X.columns.tolist())

  return X_train, X_val, y_train, y_val

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

def prep_data_cross_validation(df, num_folds=5):
  X = df.copy().drop(columns=['Quest16_MCQ160B'])
  y = df['Quest16_MCQ160B']

  y = y.replace({2: 0})
  y = y.replace({9: 0})

  y = y.astype(int)

  kf = KFold(n_splits=num_folds, shuffle=True, random_state=37)

  for train_index, val_index in kf.split(X):
    X_train, X_val, y_train, y_val = X.iloc[train_index, :], X.iloc[val_index, :], y.iloc[train_index], y.iloc[val_index]

    scaler = StandardScaler()
    X_train.iloc[:, 28:] = scaler.fit_transform(X_train.iloc[:, 28:])
    X_val.iloc[:, 28:] = scaler.transform(X_val.iloc[:, 28:])

    X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
    X_val = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)

    yield X_train, X_val, y_train, y_val


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

class CNNModel(BaseEstimator, ClassifierMixin):
  def __init__(
      self,
      input_shape,
      layer_metadata,
      convolution_part=['conv1d'],
      intermediate_part=['flatten'],
      dense_part=['dense'],
      convolution_hyperparameter=[{'filters':16, 'kernel_size':3, 'activation':'relu'}],
      intermediate_hyperparameter=[{}],
      dense_hyperparameter=[{'units':32, 'activation':'relu'}],
      optimizer='adam'
  ):
    self.input_shape = input_shape
    self.layer_metadata = layer_metadata
    self.convolution_part = convolution_part
    self.intermediate_part = intermediate_part
    self.dense_part = dense_part
    self.convolution_hyperparameter = convolution_hyperparameter
    self.intermediate_hyperparameter = intermediate_hyperparameter
    self.dense_hyperparameter = dense_hyperparameter
    self.optimizer = optimizer
    self.model = self.build_model(input_shape, convolution_part, intermediate_part, dense_part, convolution_hyperparameter, intermediate_hyperparameter, dense_hyperparameter, optimizer)



  def build_model(
      self,
      input_shape,
      convolution_part,
      intermediate_part,
      dense_part,
      convolution_hyperparameter,
      intermediate_hyperparameter,
      dense_hyperparameter,
      optimizer,
  ):

    def fix_hyperparameter_type(layer, hyperparameter, layer_metadata, warnings=True):
      for layer_info in layer_metadata:
        if(layer == layer_info['layer']):
          for params in layer_info['hyperparameter']:
            val = hyperparameter.get(params['param'], params['default'])
            if(type(val) != params['type']):
              if(warnings): print("Hyperparameter ", params['param'], " Have Invalid Data Type! Using Default..")
              hyperparameter[params['param']] = params['default']
            else:
              hyperparameter[params['param']] = val
        else:
          pass
      return hyperparameter

    model = models.Sequential()
    model.add(layers.Input(input_shape))

    # Convolution Part
    for layer, hyperparameter in zip(convolution_part, convolution_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'conv1d'):
        model.add(layers.Conv1D(filters=hyperparameter['filters'], kernel_size=hyperparameter['kernel_size'], activation=hyperparameter['activation']))
      elif(layer == 'maxpooling1d'):
        model.add(layers.MaxPooling1D(pool_size=hyperparameter['pool_size']))
      else:
        print("'Convolution Part' Layer Invalid!")
        return None

    # Intermediate Part
    for layer, hyperparameter in zip(intermediate_part, intermediate_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'flatten'):
        model.add(layers.Flatten())
      else:
        print("'Intermediate Part' Layer Invalid!")
        return None

    # Dense Part
    for layer, hyperparameter in zip(dense_part, dense_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'dense'):
        model.add(layers.Dense(units=hyperparameter['units'], activation=hyperparameter['activation']))
      else:
        print("'Dense Part' Layer Invalid!")
        return None

    model.add(layers.Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

  def fit(self, X_train, y_train, X_test, y_test, epochs, verbose=1):
    self.model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=verbose)

  def score(self, X, y):
    loss, accuracy = self.model.evaluate(X, y, verbose=0)
    loss_inverse = 1/(loss+1e-20)

    return loss_inverse

  def evaluate(self, X, y, verbose=1):
    # Evaluate the model on the test set
    loss, accuracy = self.model.evaluate(X, y, verbose=0)

    if(verbose != 0):
      print(f"Loss: %.3f%%" % (loss*100) )
      print(f"Accuracy: %.3f%%" % (accuracy*100) )

    return loss, accuracy

  def summary(self):
    self.model.summary()

  def predict(self, X):
    return self.model.predict(X, verbose=0)

  def save(self, path, save_format='keras'):
    return self.model.save(path, save_format)

In [ ]:
layer_metadata = [
  {'layer': 'conv1d', 'hyperparameter': [
      {'param':'filters', 'type':int, 'default':16},
      {'param':'kernel_size', 'type':int, 'default':3},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  },
  {'layer': 'maxpooling1d', 'hyperparameter': [
      {'param':'pool_size', 'type':int, 'default':2}
    ]
  },
  {'layer': 'flatten', 'hyperparameter': [

    ]
  },
  {'layer': 'dense', 'hyperparameter': [
      {'param':'units', 'type':int, 'default':8},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  }
]

In [ ]:
tf.random.set_seed(37)
random.seed(37)

X_train, X_val, y_train, y_val = prep_data(df_raw)

model_schema = {}

# must include
model_schema['input_shape'] = X_train.shape[1:]
model_schema['layer_metadata'] = layer_metadata

# option
model_schema['convolution_part'] = ['conv1d', 'maxpooling1d', 'conv1d', 'maxpooling1d']
model_schema['convolution_hyperparameter'] =  [{'filters':1, 'kernel_size':3}, {}, {'filters':64, 'kernel_size':1, 'activation':'relu'}, {}]

model_schema['dense_part'] = ['dense', 'dense', 'dense']
model_schema['dense_hyperparameter'] =  [ {'units':2, 'activation':'relu'}, {'units':64, 'activation':'relu'}, {'units':16, 'activation':'relu'}]

model = CNNModel(**model_schema)

model.summary()

model.fit(X_train, y_train, X_val, y_val, epochs=5)
model.evaluate(X_val, y_val)

Train:  (6477, 85, 1)   (6477,)
Val:  (2777, 85, 1)   (2777,)
Column Used:  ['Dieta1_DR1TKCAL', 'Exami2_BMXBMI', 'Dieta1_DR1TCARB', 'Dieta1_DR1TFIBE', 'Dieta1_DR1TSFAT', 'Dieta1_DR1TTFAT', 'Dieta1_DR1TPFAT', 'Dieta1_DR1TCALC', 'Quest6_DED125', 'Dieta1_DR1TMFAT', 'Quest21_SLQ330', 'Quest21_SLD013', 'Quest21_SLQ300', 'Quest19_PAQ635', 'Quest19_PAD615', 'Quest19_PAQ640', 'Quest21_SLQ320', 'Quest21_SLD012', 'Dieta1_DRDINT', 'Quest19_PAQ610', 'Quest19_PAQ655', 'Dieta1_DR1DAY', 'Dieta1_DR1TPROT', 'Quest6_DED120', 'Quest19_PAD645', 'Dieta1_DR1TCHOL', 'Quest19_PAD660', 'Dieta1_DR1TSUGR', 'Demog1_RIDRETH3', 'Demog1_DMDMARTL', 'Quest14_INQ012', 'Demog1_DMDHHSZA', 'Quest15_KIQ026', 'Quest9_DLQ050', 'Demog1_DMDHHSIZ', 'Exami1_BPXPULS', 'Quest18_OCQ210', 'Demog1_INDIN2', 'Demog1_RIDAGEYR', 'Quest15_KIQ022', 'Quest20_PFQ061C', 'Quest4_CBD111', 'Quest7_DIQ010', 'Quest14_IND235', 'Demog1_DMDEDUC', 'Quest10_ECQ020', 'Labor1_LBDTCSI', 'Quest17_DPQ020', 'Quest3_CDQ008', 'Quest14_INQ020', 'Demog1_DMDFMSIZ

(0.08677980303764343, 0.9769535660743713)

In [ ]:
tf.random.set_seed(37)
random.seed(37)

model_schema = {}

# must include
model_schema['input_shape'] = X_train.shape[1:]
model_schema['layer_metadata'] = layer_metadata

# option
model_schema['convolution_part'] = ['conv1d', 'maxpooling1d', 'conv1d', 'maxpooling1d']
model_schema['convolution_hyperparameter'] =  [{'filters':1, 'kernel_size':3}, {}, {'filters':64, 'kernel_size':1, 'activation':'relu'}, {}]

model_schema['dense_part'] = ['dense', 'dense', 'dense']
model_schema['dense_hyperparameter'] =  [ {'units':2, 'activation':'relu'}, {'units':64, 'activation':'relu'}, {'units':16, 'activation':'relu'}]

loss = []
accuracy = []
train_loss = []
train_accuracy = []

for X_train, X_val, y_train, y_val in prep_data_cross_validation(df_raw, num_folds=4):
  print(y_train.value_counts())
  print(y_val.value_counts())

  model = CNNModel(**model_schema)

  model.fit(X_train, y_train, X_val, y_val, epochs=5)
  _loss, _accuracy = model.evaluate(X_val, y_val)
  loss.append(_loss)
  accuracy.append(_accuracy)
  _train_loss, _train_accuracy = model.evaluate(X_train, y_train, verbose=0)
  train_loss.append(_train_loss)
  train_accuracy.append(_train_accuracy)

print()
model.summary()

print(loss)
print(accuracy)

print("Training Loss: " + str(np.mean(train_loss)*100) + "%")
print("Training Accuracy: " + str(np.mean(train_accuracy)*100) + "%")

print("Loss: " + str(np.mean(loss)*100) + "%")
print("Accuracy: " + str(np.mean(accuracy)*100) + "%")

Quest16_MCQ160B
0    4035
1    2905
Name: count, dtype: int64
Quest16_MCQ160B
0    1333
1     981
Name: count, dtype: int64
Epoch 1/5
217/217 [==============================] - 6s 14ms/step - loss: 0.2272 - accuracy: 0.9378 - val_loss: 0.1117 - val_accuracy: 0.9667
Epoch 2/5
217/217 [==============================] - 2s 8ms/step - loss: 0.1201 - accuracy: 0.9641 - val_loss: 0.1308 - val_accuracy: 0.9602
Epoch 3/5
217/217 [==============================] - 2s 11ms/step - loss: 0.1086 - accuracy: 0.9673 - val_loss: 0.0867 - val_accuracy: 0.9745
Epoch 4/5
217/217 [==============================] - 3s 15ms/step - loss: 0.0944 - accuracy: 0.9710 - val_loss: 0.0760 - val_accuracy: 0.9780
Epoch 5/5
217/217 [==============================] - 2s 7ms/step - loss: 0.0879 - accuracy: 0.9719 - val_loss: 0.0766 - val_accuracy: 0.9797
Loss: 7.661%
Accuracy: 97.969%
Quest16_MCQ160B
0    4026
1    2914
Name: count, dtype: int64
Quest16_MCQ160B
0    1342
1     972
Name: count, dtype: int64
Epoch 1/5
217

In [ ]:
tf.random.set_seed(37)
random.seed(37)

log = None

input_shape = X_train.shape[1:]
output_shape = 1

param_grid = {
  'n_conv': [1, 3],
  'n_dense':[0, 1, 2],
  'conv_layer':['conv1d', 'maxpooling1d'],
  'dense_layer':['dense'],
  'filters':[2, 16, 64],
  'kernel_size':[1, 3],
  'units': [2, 16, 64],
  'activation': ['linear', 'relu']
}

def save_result(save_state, target_path):
  log.to_csv(target_path, index=False)

def get_schema_combination(part_name, n_layer, layer_values, layer_metadata, param_grid):
  schema_combination = []
  for n in n_layer:
    part_combination=itertools.product(layer_values, repeat=n)

    for part in part_combination:
      hyperparameters = []

      for selected_layer in part:

        # Get selected layers's possible hyperparameters
        hyperparameter_values = {}
        for layer in layer_metadata:
          if(selected_layer == layer['layer']):
            for params in layer['hyperparameter']:
              hyperparameter_values[params['param']] = [params['default']]
              if params['param'] in list(param_grid):
                hyperparameter_values[params['param']] = param_grid[params['param']]

        # get hyperparameter combination to grid search
        hyperparameter_combination = []
        for combination in list(itertools.product(*hyperparameter_values.values())):
          layer_hyperparameter = {}
          for key, val in zip(hyperparameter_values.keys(), combination):
            layer_hyperparameter[key] = val
          hyperparameter_combination.append(layer_hyperparameter)

        hyperparameters.append(hyperparameter_combination)

      for hyperparameter in list(itertools.product(*hyperparameters)):
        schema = {part_name+'_part': list(part), part_name+'_hyperparameter': list(hyperparameter)}
        schema_combination.append(schema)

  return schema_combination

conv_shcema = get_schema_combination(part_name='convolution',
                       n_layer = param_grid.get('n_conv', [1]),
                       layer_values = param_grid.get('conv_layer', ['conv1d']),
                       layer_metadata=layer_metadata,
                       param_grid=param_grid)

dense_schema = get_schema_combination(part_name='dense',
                       n_layer = param_grid.get('n_dense', [1]),
                       layer_values = param_grid.get('dense_layer', ['dense']),
                       layer_metadata=layer_metadata,
                       param_grid=param_grid)

model_schema_combination = []
for schema in itertools.product(conv_shcema, dense_schema):
  model_schema = {}
  for schema_dict in schema:
    model_schema.update(schema_dict)

  model_schema_combination.append(model_schema)

for i, model_schema in enumerate(model_schema_combination):

  model_schema['input_shape'] = input_shape
  model_schema['layer_metadata'] = layer_metadata

  loss = []
  accuracy = []
  train_loss = []
  train_accuracy = []

  model = CNNModel(**model_schema)
  model.summary()

  print("Training Model...")
  for X_train, X_val, y_train, y_val in prep_data_cross_validation(df_raw, num_folds=4):

    model = CNNModel(**model_schema)

    model.fit(X_train, y_train, X_val, y_val, epochs=10, verbose=0)
    _loss, _accuracy = model.evaluate(X_val, y_val, verbose=0)
    loss.append(_loss)
    accuracy.append(_accuracy)
    _train_loss, _train_accuracy = model.evaluate(X_train, y_train, verbose=0)
    train_loss.append(_train_loss)
    train_accuracy.append(_train_accuracy)

  print(f"Training Loss: %.3f%%" %  (np.mean(train_loss)*100) )
  print(f"Training Accuracy: %.3f%%" % (np.mean(train_accuracy)*100) )
  print(train_loss)
  print(train_accuracy)
  print(f"Loss: %.3f%%" %  (np.mean(loss)*100) )
  print(f"Accuracy: %.3f%%" % (np.mean(accuracy)*100) )
  print(loss)
  print(accuracy)

  loss, accuracy = np.mean(loss), np.mean(accuracy)
  train_loss, train_accuracy = np.mean(train_loss), np.mean(train_accuracy)

  save_state = {}
  save_state.update(model_schema)
  save_state.update({'loss': loss, 'accuracy': accuracy})
  save_state.update({'train_loss': train_loss, 'train_accuracy': train_accuracy})

  log = pd.concat([log, pd.DataFrame({key: [value] for key, value in save_state.items()})])

  save_result(log, 'hyperparameter_tuning_log.csv')

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_52 (Conv1D)          (None, 85, 2)             4         
                                                                 
 flatten_26 (Flatten)        (None, 170)               0         
                                                                 
 dense_104 (Dense)           (None, 2)                 342       
                                                                 
Total params: 346 (1.35 KB)
Trainable params: 346 (1.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Training Model...
Training Loss: 6.842%
Training Accuracy: 97.875%
[0.07030276954174042, 0.06886427104473114, 0.06640252470970154, 0.0681098997592926]
[0.9768011569976807, 0.98069167137146, 0.9798299670219421, 0.9776689410209656]
Loss: 7.602%
Accuracy: 97.601%
[0.07098362594842911, 0.07

KeyboardInterrupt: 

In [ ]:
result = pd.read_csv('hyperparameter_tuning_log.csv')

result

,convolution_part,convolution_hyperparameter,dense_part,dense_hyperparameter,input_shape,layer_metadata,loss,accuracy,train_loss,train_accuracy
0,['conv1d'],"[{'filters': 2, 'kernel_size': 1, 'activation'...",[],[],"(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.076018,0.97601,0.068420,0.978748
1,['conv1d'],"[{'filters': 2, 'kernel_size': 1, 'activation'...",['dense'],"[{'units': 2, 'activation': 'linear'}]","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.076776,0.97601,0.068798,0.978424


In [ ]:
best_index = result['loss'].idxmin()

# Get the row with the highest value in column 'A'
best_model = result.loc[best_index]

print(best_model)
print(best_model.tolist())

convolution_part                                                     ['conv1d']
convolution_hyperparameter    [{'filters': 2, 'kernel_size': 1, 'activation'...
dense_part                                                                   []
dense_hyperparameter                                                         []
input_shape                                                             (85, 1)
layer_metadata                [{'layer': 'conv1d', 'hyperparameter': [{'para...
loss                                                                   0.076018
accuracy                                                                0.97601
train_loss                                                              0.06842
train_accuracy                                                         0.978748
Name: 0, dtype: object
["['conv1d']", "[{'filters': 2, 'kernel_size': 1, 'activation': 'linear'}]", '[]', '[]', '(85, 1)', "[{'layer': 'conv1d', 'hyperparameter': [{'param': 'filters', 'type': <class 